In [13]:
import gensim
from nltk import word_tokenize, sent_tokenize
import multiprocessing
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from collections import Counter
from statistics import mean

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('clean_tweets_update.csv', usecols=['tweets', 'pos_lems'])

In [4]:
target_df = pd.read_csv('train.csv', usecols=['target'])
target = list(target_df.target)

In [5]:
print(len(target))

7613


In [6]:
train_df, test_df = df.iloc[:len(target_df), :], df.iloc[len(target_df):, :] 
train_df['target'] = target
train_df.drop_duplicates(subset=['pos_lems'], keep='last', inplace=True)

<ipython-input-6-c9a03ecca595>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['target'] = target
<ipython-input-6-c9a03ecca595>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop_duplicates(subset=['pos_lems'], keep='last', inplace=True)


In [7]:
train_length = len(train_df)
print(train_length)

6851


In [8]:
test_length = len(test_df)
print(test_length)

3263


In [802]:
texts = [train_df.pos_lems, test_df.pos_lems]
tweets = pd.concat(texts)
len(tweets)

10114

In [11]:
train_tweets = train_df.tweets.replace('at_user', '')
train_tweets = train_tweets.replace('url_here', '')
test_tweets = test_df.tweets.replace('at_user', '')
test_tweets = test_tweets.replace('url_here', '')

In [15]:
def tokenize_text(text):
    tokens = []
    for sent in sent_tokenize(text):
        for word in word_tokenize(sent):
            if len(word)>1:
                tokens.append(word.lower())
    return tokens

In [16]:
train = train_tweets.apply(tokenize_text)
test = test_tweets.apply(tokenize_text)

## pretrained word_vectors model

In [41]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import keras

In [419]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

In [24]:
X_train, X_test, y_train, y_test = train_test_split(train, train_df.target, test_size=0.2, random_state=88)

train_df.lem = train_df.lem.replace('at_user', '')
test_df.lem = test_df.lem.replace('at_user', '')
train_df.lem = train_df.lem.replace('url_here', '')
test_df.lem = test_df.lem.replace('url_here', '')

In [60]:
from collections import Counter
from statistics import mean

word_dict = Counter(test.sum())
# count number of words in corpus
num_words_test = sum(word_dict[w] for w in word_dict)
print(f'There are {num_words_test} words in the unseen dataset')

# count number of unique words in corpus
word_count_sorted = [(value, key) for key, value in word_dict.items()]
word_count_sorted.sort()
vocab_size_test = len(word_count_sorted)
print(f'There are {vocab_size_test} unique words in the unseen dataset')
print("max len of training tweets",max([len(x) for x in test]))
print("min len of training tweets",min([len(x) for x in test]))
print("mean len of training tweets", mean([len(x) for x in test]))


There are 37607 words in the unseen dataset
There are 7008 unique words in the unseen dataset
max len of training tweets 27
min len of training tweets 1
mean len of training tweets 11.52528348145878


In [61]:
word_dict = Counter(X_train.sum())
# count number of words in corpus
num_words = sum(word_dict[w] for w in word_dict)
print(f'There are {num_words} words in the training dataset')

# count number of unique words in corpus
word_count_sorted = [(value, key) for key, value in word_dict.items()]
word_count_sorted.sort()
vocab_size = len(word_count_sorted)
maxlen = max([len(x) for x in X_train])
print(f'There are {vocab_size} unique words in the training dataset')
print("max len of training tweets",max([len(x) for x in X_train]))
print("max len of test tweets",max([len(x) for x in X_test]))
print("min len of training tweets",min([len(x) for x in X_train]))
print("min len of test tweets",min([len(x) for x in X_test]))
print("mean len of training tweets", mean([len(x) for x in X_train]))
print("mean len of test tweets",mean([len(x) for x in X_test]))

There are 63060 words in the training dataset
There are 9689 unique words in the training dataset
max len of training tweets 28
max len of test tweets 27
min len of training tweets 1
min len of test tweets 1
mean len of training tweets 11.507299270072993
mean len of test tweets 11.412107950401166


# embedding size : 50

In [62]:
# load the word embedding file from local
embeddings_index = dict()
f = open('utils//glove.6B.50d.txt', encoding="utf8")
# iterate through text, splitting, obtain index & float32 for words
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
# close embedding file from local to save on memory
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [63]:
tokenizer = text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)
tokenized_train = tokenizer.texts_to_sequences(X_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

tokenized_test = tokenizer.texts_to_sequences(X_test)
x_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [34]:
tokenized_unseen_test = tokenizer.texts_to_sequences(test)
unseen_test = sequence.pad_sequences(tokenized_unseen_test, maxlen=maxlen)

In [35]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
n_words = num_words
#change below line if computing normal stats is too slow
embedding_matrix = np.random.normal(emb_mean, emb_std,(n_words, embed_size))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

C:\Users\root\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


### callbacks

In [38]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.01, min_lr=0.06)
early_stop = EarlyStopping(monitor='loss', min_delta=0.01, patience=5, verbose=1)

In [39]:
batch_size = 300
epochs = 100
embed_size = 50

#### adam

In [42]:
#Defining Neural Network
model = Sequential()
#Non-trainable embedding layer
model.add(Embedding(n_words, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(128, return_sequences = True , recurrent_dropout = 0.4 , dropout = 0.4))
model.add(LSTM(64, return_sequences = True , recurrent_dropout = 0.2 , dropout = 0.3))
model.add(LSTM(32, recurrent_dropout = 0.15 , dropout = 0.25))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.03), loss='binary_crossentropy', metrics=['accuracy'])

In [43]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs = epochs , callbacks = [learning_rate_reduction])

Epoch 1/100
19/19 [==============================] - 3s 183ms/step - loss: 0.7099 - accuracy: 0.5812 - val_loss: 0.6725 - val_accuracy: 0.6010
Epoch 2/100
19/19 [==============================] - 3s 138ms/step - loss: 0.6775 - accuracy: 0.5905 - val_loss: 0.6698 - val_accuracy: 0.6010
Epoch 3/100
19/19 [==============================] - 3s 147ms/step - loss: 0.6700 - accuracy: 0.5905 - val_loss: 0.6267 - val_accuracy: 0.6010
Epoch 4/100
19/19 [==============================] - 3s 153ms/step - loss: 0.6173 - accuracy: 0.6396 - val_loss: 0.5725 - val_accuracy: 0.7535
Epoch 5/100
19/19 [==============================] - 3s 154ms/step - loss: 0.5453 - accuracy: 0.7547 - val_loss: 0.5147 - val_accuracy: 0.7593
Epoch 6/100
19/19 [==============================] - 3s 159ms/step - loss: 0.5604 - accuracy: 0.7303 - val_loss: 0.5102 - val_accuracy: 0.7841
Epoch 7/100
19/19 [==============================] - 3s 159ms/step - loss: 0.5164 - accuracy: 0.7681 - val_loss: 0.4617 - val_accuracy: 0.7907

Epoch 58/100
19/19 [==============================] - 3s 179ms/step - loss: 0.5217 - accuracy: 0.7524 - val_loss: 0.4919 - val_accuracy: 0.7921
Epoch 59/100
19/19 [==============================] - 3s 177ms/step - loss: 0.5254 - accuracy: 0.7429 - val_loss: 0.4715 - val_accuracy: 0.7929
Epoch 60/100
19/19 [==============================] - 3s 173ms/step - loss: 0.5293 - accuracy: 0.7500 - val_loss: 0.5000 - val_accuracy: 0.7899
Epoch 61/100
19/19 [==============================] - 3s 175ms/step - loss: 0.5249 - accuracy: 0.7524 - val_loss: 0.4723 - val_accuracy: 0.7834
Epoch 62/100
19/19 [==============================] - 3s 182ms/step - loss: 0.5161 - accuracy: 0.7630 - val_loss: 0.4681 - val_accuracy: 0.7863
Epoch 63/100
19/19 [==============================] - 3s 177ms/step - loss: 0.5110 - accuracy: 0.7635 - val_loss: 0.4590 - val_accuracy: 0.8031
Epoch 64/100
19/19 [==============================] - 3s 175ms/step - loss: 0.5195 - accuracy: 0.7555 - val_loss: 0.4543 - val_accuracy:

In [59]:
ynew = np.argmax(model.predict(unseen_test), axis=-1)
submission = pd.read_csv('test.csv', usecols=['id'])
submission['target'] = ynew
submission.to_csv('predictions.csv')

#### rmsprop

In [46]:
#Defining Neural Network
model = Sequential()
#Non-trainable embedding layer
model.add(Embedding(n_words, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(256 , return_sequences = True , recurrent_dropout = 0.5 , dropout = 0.5))
model.add(LSTM(128, recurrent_dropout = 0.6 , dropout = 0.3))
#model.add(Dense(32, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
#model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.RMSprop(), loss='binary_crossentropy', metrics=['accuracy'])

In [47]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs = epochs)

Epoch 1/100
19/19 [==============================] - 7s 351ms/step - loss: 0.6059 - accuracy: 0.6619 - val_loss: 0.4849 - val_accuracy: 0.7826
Epoch 2/100
19/19 [==============================] - 7s 357ms/step - loss: 0.5142 - accuracy: 0.7588 - val_loss: 0.4663 - val_accuracy: 0.7921
Epoch 3/100
19/19 [==============================] - 7s 357ms/step - loss: 0.5042 - accuracy: 0.7693 - val_loss: 0.4729 - val_accuracy: 0.7885
Epoch 4/100
19/19 [==============================] - 7s 383ms/step - loss: 0.4946 - accuracy: 0.7723 - val_loss: 0.4796 - val_accuracy: 0.7826
Epoch 5/100
19/19 [==============================] - 7s 367ms/step - loss: 0.4941 - accuracy: 0.7692 - val_loss: 0.4648 - val_accuracy: 0.8016
Epoch 6/100
19/19 [==============================] - 7s 369ms/step - loss: 0.4946 - accuracy: 0.7699 - val_loss: 0.4468 - val_accuracy: 0.8053
Epoch 7/100
19/19 [==============================] - 7s 372ms/step - loss: 0.4857 - accuracy: 0.7712 - val_loss: 0.4596 - val_accuracy: 0.8001

Epoch 58/100
19/19 [==============================] - 7s 393ms/step - loss: 0.3391 - accuracy: 0.8544 - val_loss: 0.4547 - val_accuracy: 0.8082
Epoch 59/100
19/19 [==============================] - 8s 396ms/step - loss: 0.3349 - accuracy: 0.8529 - val_loss: 0.4806 - val_accuracy: 0.8118
Epoch 60/100
19/19 [==============================] - 8s 396ms/step - loss: 0.3355 - accuracy: 0.8527 - val_loss: 0.5119 - val_accuracy: 0.8031
Epoch 61/100
19/19 [==============================] - 8s 400ms/step - loss: 0.3322 - accuracy: 0.8595 - val_loss: 0.5431 - val_accuracy: 0.7929
Epoch 62/100
19/19 [==============================] - 8s 398ms/step - loss: 0.3232 - accuracy: 0.8586 - val_loss: 0.4816 - val_accuracy: 0.8082
Epoch 63/100
19/19 [==============================] - 8s 399ms/step - loss: 0.3251 - accuracy: 0.8615 - val_loss: 0.4883 - val_accuracy: 0.8125
Epoch 64/100
19/19 [==============================] - 8s 398ms/step - loss: 0.3187 - accuracy: 0.8646 - val_loss: 0.4965 - val_accuracy:

In [694]:
ynew = model.predict_classes(unseen_test)
submission = pd.read_csv('test.csv', usecols=['id'])
submission['target'] = ynew
submission.to_csv('predictions1.csv')

# embedding size : 100

#### load embeddings from local

In [64]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('utils//glove.6B.100d.txt', encoding="utf8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [65]:
# fit embeddings to data

tokenizer = text.Tokenizer(num_words=vocab_size) # vocab size of training data
tokenizer.fit_on_texts(X_train)
tokenized_train = tokenizer.texts_to_sequences(X_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

tokenized_test = tokenizer.texts_to_sequences(X_test)
x_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [66]:
tokenized_unseen_test = tokenizer.texts_to_sequences(test_df.texts)
unseen_test = sequence.pad_sequences(tokenized_unseen_test, maxlen=maxlen)

AttributeError: 'DataFrame' object has no attribute 'texts'

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
n_words = num_words
#change below line if computing normal stats is too slow
embedding_matrix = np.random.normal(emb_mean, emb_std,(n_words, embed_size))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [67]:
maxlen

28

In [53]:
batch_size = 300
epochs = 150
embed_size = 100

#### adam

In [54]:
#Defining Neural Network
model = Sequential()
#Non-trainable embedding layer
model.add(Embedding(n_words, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(256, return_sequences = True, recurrent_dropout = 0.4 , dropout = 0.4))
model.add(LSTM(128, return_sequences = True , recurrent_dropout = 0.35 , dropout = 0.35))
model.add(LSTM(64, return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(32, dropout = 0.4))
#model.add(Dense(25, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
#model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy']) 

In [55]:
history = model.fit(x_train, y_train, batch_size = 500, validation_data = (x_test,y_test) , epochs = 120 , callbacks = [learning_rate_reduction], shuffle=True)

Epoch 1/120
11/11 [==============================] - 10s 915ms/step - loss: 0.6907 - accuracy: 0.5896 - val_loss: 0.6748 - val_accuracy: 0.6010
Epoch 2/120
11/11 [==============================] - 9s 777ms/step - loss: 0.6775 - accuracy: 0.5905 - val_loss: 0.6673 - val_accuracy: 0.6010
Epoch 3/120
11/11 [==============================] - 9s 784ms/step - loss: 0.6756 - accuracy: 0.5903 - val_loss: 0.6745 - val_accuracy: 0.6010
Epoch 4/120
11/11 [==============================] - 9s 808ms/step - loss: 0.6737 - accuracy: 0.5978 - val_loss: 0.6727 - val_accuracy: 0.6010
Epoch 5/120
11/11 [==============================] - 9s 827ms/step - loss: 0.6771 - accuracy: 0.5991 - val_loss: 0.6748 - val_accuracy: 0.6010
Epoch 6/120
11/11 [==============================] - 9s 843ms/step - loss: 0.6774 - accuracy: 0.5905 - val_loss: 0.6738 - val_accuracy: 0.6010
Epoch 7/120
11/11 [==============================] - 9s 849ms/step - loss: 0.6779 - accuracy: 0.5905 - val_loss: 0.6733 - val_accuracy: 0.601

11/11 [==============================] - 12s 1s/step - loss: 0.3035 - accuracy: 0.8816 - val_loss: 0.4623 - val_accuracy: 0.8191
Epoch 59/120
11/11 [==============================] - 12s 1s/step - loss: 0.3064 - accuracy: 0.8799 - val_loss: 0.4710 - val_accuracy: 0.8133
Epoch 60/120
11/11 [==============================] - 12s 1s/step - loss: 0.3139 - accuracy: 0.8745 - val_loss: 0.5121 - val_accuracy: 0.7914
Epoch 61/120
11/11 [==============================] - 12s 1s/step - loss: 0.2979 - accuracy: 0.8878 - val_loss: 0.4587 - val_accuracy: 0.8140
Epoch 62/120
11/11 [==============================] - 12s 1s/step - loss: 0.2928 - accuracy: 0.8838 - val_loss: 0.4652 - val_accuracy: 0.8089
Epoch 63/120
11/11 [==============================] - 12s 1s/step - loss: 0.2868 - accuracy: 0.8885 - val_loss: 0.5180 - val_accuracy: 0.8074
Epoch 64/120
11/11 [==============================] - 12s 1s/step - loss: 0.2796 - accuracy: 0.8929 - val_loss: 0.5083 - val_accuracy: 0.8031
Epoch 65/120
11/11 

Epoch 116/120
11/11 [==============================] - 11s 1s/step - loss: 0.2426 - accuracy: 0.9011 - val_loss: 0.5602 - val_accuracy: 0.7994
Epoch 117/120
11/11 [==============================] - 11s 1s/step - loss: 0.2379 - accuracy: 0.9033 - val_loss: 0.5762 - val_accuracy: 0.8045
Epoch 118/120
11/11 [==============================] - 11s 1s/step - loss: 0.2400 - accuracy: 0.9060 - val_loss: 0.5610 - val_accuracy: 0.8125
Epoch 119/120
11/11 [==============================] - 11s 1s/step - loss: 0.2390 - accuracy: 0.9075 - val_loss: 0.5657 - val_accuracy: 0.8016
Epoch 120/120
11/11 [==============================] - 11s 1s/step - loss: 0.2396 - accuracy: 0.9066 - val_loss: 0.5528 - val_accuracy: 0.8001


In [768]:
ynew = model.predict_classes(unseen_test)
submission = pd.read_csv('test.csv', usecols=['id'])
submission['target'] = ynew
submission.to_csv('predictions2.csv')

In [747]:
model.summary()

Model: "sequential_124"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_124 (Embedding)    (None, 32, 50)            4432250   
_________________________________________________________________
lstm_254 (LSTM)              (None, 32, 128)           91648     
_________________________________________________________________
lstm_255 (LSTM)              (None, 32, 64)            49408     
_________________________________________________________________
lstm_256 (LSTM)              (None, 32)                12416     
_________________________________________________________________
dense_286 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_287 (Dense)            (None, 1)                 17        
Total params: 4,586,267
Trainable params: 154,017
Non-trainable params: 4,432,250
____________________________________

In [ ]:
### rmsprop

In [56]:
#Defining Neural Network
model = Sequential()
#Non-trainable embedding layer
model.add(Embedding(n_words, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
#LSTM 
model.add(LSTM(256, return_sequences = True, recurrent_dropout = 0.4 , dropout = 0.4))
model.add(LSTM(128, return_sequences = True , recurrent_dropout = 0.35 , dropout = 0.35))
model.add(LSTM(64, return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(32, dropout = 0.4))
#model.add(Dense(25, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
#model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.RMSprop(), loss='binary_crossentropy', metrics=['accuracy']) 

In [57]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (x_test,y_test) , epochs = epochs )

Epoch 1/150
19/19 [==============================] - 20s 1s/step - loss: 0.6718 - accuracy: 0.5949 - val_loss: 0.5992 - val_accuracy: 0.6842
Epoch 2/150
19/19 [==============================] - 19s 999ms/step - loss: 0.5806 - accuracy: 0.7157 - val_loss: 0.5106 - val_accuracy: 0.7834
Epoch 3/150
19/19 [==============================] - 19s 989ms/step - loss: 0.5252 - accuracy: 0.7599 - val_loss: 0.5206 - val_accuracy: 0.7360
Epoch 4/150
19/19 [==============================] - 19s 987ms/step - loss: 0.5095 - accuracy: 0.7670 - val_loss: 0.4778 - val_accuracy: 0.7863
Epoch 5/150
19/19 [==============================] - 19s 992ms/step - loss: 0.4952 - accuracy: 0.7724 - val_loss: 0.4665 - val_accuracy: 0.7921
Epoch 6/150
19/19 [==============================] - 19s 999ms/step - loss: 0.4909 - accuracy: 0.7772 - val_loss: 0.4667 - val_accuracy: 0.7965
Epoch 7/150
19/19 [==============================] - 19s 995ms/step - loss: 0.4795 - accuracy: 0.7838 - val_loss: 0.4987 - val_accuracy: 0.

Epoch 58/150
19/19 [==============================] - 19s 994ms/step - loss: 0.2426 - accuracy: 0.8982 - val_loss: 0.5955 - val_accuracy: 0.8162
Epoch 59/150
19/19 [==============================] - 19s 996ms/step - loss: 0.2329 - accuracy: 0.9075 - val_loss: 0.5838 - val_accuracy: 0.7892
Epoch 60/150
19/19 [==============================] - 19s 990ms/step - loss: 0.2395 - accuracy: 0.9027 - val_loss: 0.5701 - val_accuracy: 0.7929
Epoch 61/150
19/19 [==============================] - 19s 991ms/step - loss: 0.2221 - accuracy: 0.9095 - val_loss: 0.6299 - val_accuracy: 0.8031
Epoch 62/150
19/19 [==============================] - 19s 990ms/step - loss: 0.2355 - accuracy: 0.9046 - val_loss: 0.5711 - val_accuracy: 0.7929
Epoch 63/150
19/19 [==============================] - 19s 989ms/step - loss: 0.2220 - accuracy: 0.9091 - val_loss: 0.6319 - val_accuracy: 0.7710
Epoch 64/150
19/19 [==============================] - 19s 986ms/step - loss: 0.2109 - accuracy: 0.9201 - val_loss: 0.6219 - val_ac

19/19 [==============================] - 19s 992ms/step - loss: 0.0997 - accuracy: 0.9608 - val_loss: 0.8941 - val_accuracy: 0.7972
Epoch 115/150
19/19 [==============================] - 19s 990ms/step - loss: 0.0889 - accuracy: 0.9688 - val_loss: 0.8675 - val_accuracy: 0.7739
Epoch 116/150
19/19 [==============================] - 19s 989ms/step - loss: 0.0891 - accuracy: 0.9668 - val_loss: 0.9131 - val_accuracy: 0.7943
Epoch 117/150
19/19 [==============================] - 19s 985ms/step - loss: 0.0994 - accuracy: 0.9604 - val_loss: 0.8572 - val_accuracy: 0.8067
Epoch 118/150
19/19 [==============================] - 19s 992ms/step - loss: 0.0875 - accuracy: 0.9670 - val_loss: 0.8300 - val_accuracy: 0.7805
Epoch 119/150
19/19 [==============================] - 19s 993ms/step - loss: 0.0875 - accuracy: 0.9684 - val_loss: 0.9693 - val_accuracy: 0.8009
Epoch 120/150
19/19 [==============================] - 19s 996ms/step - loss: 0.0957 - accuracy: 0.9646 - val_loss: 0.8224 - val_accuracy:

In [58]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 27, 100)           6306000   
_________________________________________________________________
lstm_14 (LSTM)               (None, 27, 256)           365568    
_________________________________________________________________
lstm_15 (LSTM)               (None, 27, 128)           197120    
_________________________________________________________________
lstm_16 (LSTM)               (None, 27, 64)            49408     
_________________________________________________________________
lstm_17 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dense_11 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_12 (Dense)             (None, 8)                

In [705]:
ynew = model.predict_classes(unseen_test)
submission = pd.read_csv('test.csv', usecols=['id'])
submission['target'] = ynew
submission.to_csv('predictions3.csv')